# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 參考範例，將鐵達尼的船票票號( 'Ticket' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [79]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test], ignore_index=True)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [80]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
number_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
    else:
        number_features.append(feature)
        
print(f'{len(object_features)} Object Features : {object_features}\n')
print(f'{len(number_features)} Numeric Features : {number_features}\n')

# 只留類別型欄位
df_obj = df[object_features]
df_obj = df_obj.fillna('None')
df_temp = pd.DataFrame()
for col in object_features:
    df_temp[col] = LabelEncoder().fit_transform(df_obj[col])



# numeric features
df_num = df[number_features]
df_num = df_num.fillna(df_num.mean())
df = pd.concat([df_temp,df_num], axis=1).reset_index()
train_num = train_Y.shape[0]
df.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



,index,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare
0,0,155,1,720,185,3,3,22.0,1,0,7.2500
1,1,286,0,816,106,0,1,38.0,1,0,71.2833
2,2,523,0,914,185,3,3,26.0,0,0,7.9250
3,3,422,0,65,70,3,1,35.0,1,0,53.1000
4,4,22,1,649,185,3,3,35.0,0,0,8.0500


In [94]:
# 觀察欄位相異值數量
estimator = LogisticRegression(solver='lbfgs',max_iter=250)


In [87]:
# 加上 'Ticket' 欄位的計數編碼
ticket_count = df.groupby(['Ticket'])['Name'].agg({'Ticket_count':'size'}).reset_index()
df_ticket_count = pd.merge(df,ticket_count,on=['Ticket'], how='left')
df_ticket_count = df_ticket_count.drop('index',axis=1)
df_ticket_count.head()

C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Ticket_count
0,155,1,720,185,3,3,22.0,1,0,7.2500,1
1,286,0,816,106,0,1,38.0,1,0,71.2833,2
2,523,0,914,185,3,3,26.0,0,0,7.9250,1
3,422,0,65,70,3,1,35.0,1,0,53.1000,2
4,22,1,649,185,3,3,35.0,0,0,8.0500,1


In [95]:
# 'Ticket' 計數編碼 與其他數值型欄位一起預估生存機率
train_X = df_ticket_count[:train_num]
print(f'ticket count, all features, logistic regression: {cross_val_score(estimator,train_X,train_Y,cv=5).mean()}')
df_ticket_count.head()



ticket count, all features, logistic regression: 0.7901553557059451


C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\log

,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Ticket_count
0,155,1,720,185,3,3,22.0,1,0,7.2500,1
1,286,0,816,106,0,1,38.0,1,0,71.2833,2
2,523,0,914,185,3,3,26.0,0,0,7.9250,1
3,422,0,65,70,3,1,35.0,1,0,53.1000,2
4,22,1,649,185,3,3,35.0,0,0,8.0500,1


In [98]:
# 'Ticket' 特徵雜湊 與其他數值型欄位一起預估生存機率
df_hash = df.copy()
df_hash['Ticket_hash'] = df_hash['Ticket'].map(lambda x:hash(x)%10)
train_X = df_hash[:train_num]
print(f'ticket hash, all features, logistic regression: {cross_val_score(estimator,train_X,train_Y,cv=5).mean()}')
df_hash.head()

ticket hash, all features, logistic regression: 0.7710789148258362


C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\log

,index,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Ticket_hash
0,0,155,1,720,185,3,3,22.0,1,0,7.2500,0
1,1,286,0,816,106,0,1,38.0,1,0,71.2833,6
2,2,523,0,914,185,3,3,26.0,0,0,7.9250,4
3,3,422,0,65,70,3,1,35.0,1,0,53.1000,5
4,4,22,1,649,185,3,3,35.0,0,0,8.0500,9


In [111]:
# 'Ticket' 目標均值編碼 與其他數值型欄位一起預估生存機率
df_temp = pd.concat([df[:train_num],train_Y],axis=1)
ticket_mean = df_temp.groupby(['Ticket'])['Survived'].mean().reset_index()
ticket_mean.columns = ['Ticket','Ticket_mean']
df_temp = pd.merge(df_temp, ticket_mean, on='Ticket', how='left')
df_temp = df_temp.drop('Survived',axis=1)
train_X= df_temp[:train_num]

print(f'ticket mean, all features, logistic regression: {cross_val_score(estimator,train_X,train_Y,cv=5).mean()}')
df_temp.head()

ticket mean, all features, logistic regression: 0.9820472964801951


C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\log

,index,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Ticket_mean
0,0,155,1,720,185,3,3,22.0,1,0,7.2500,0.0
1,1,286,0,816,106,0,1,38.0,1,0,71.2833,1.0
2,2,523,0,914,185,3,3,26.0,0,0,7.9250,1.0
3,3,422,0,65,70,3,1,35.0,1,0,53.1000,0.5
4,4,22,1,649,185,3,3,35.0,0,0,8.0500,0.0


# 作業2
* 承上題，三者比較效果何者最好?

<font color=red> 'Cabin'計數編碼 + 邏輯斯迴歸分數最高</font>

In [18]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression(solver='lbfgs')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7789000729487724


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [112]:
# 加上 'Cabin' 欄位的計數編碼
cabin_count = df.groupby(['Cabin'])['Name'].agg({'Cabin_count':'size'}).reset_index()
df = pd.merge(df, cabin_count, on=['Cabin'], how='left')
df.head()
# cabin_count.sort_values(by=['Cabin_count'])

C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,index,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_count
0,0,155,1,720,185,3,3,22.0,1,0,7.2500,1014
1,1,286,0,816,106,0,1,38.0,1,0,71.2833,2
2,2,523,0,914,185,3,3,26.0,0,0,7.9250,1014
3,3,422,0,65,70,3,1,35.0,1,0,53.1000,2
4,4,22,1,649,185,3,3,35.0,0,0,8.0500,1014


In [113]:
# 'Cabin'計數編碼 + 邏輯斯迴歸

train_X = df[:train_num]
print(f'Score is: {cross_val_score(estimator,train_X,train_Y,cv=5).mean()}')
train_X.head()

Score is: 0.7722342503174885


C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\log

,index,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_count
0,0,155,1,720,185,3,3,22.0,1,0,7.2500,1014
1,1,286,0,816,106,0,1,38.0,1,0,71.2833,2
2,2,523,0,914,185,3,3,26.0,0,0,7.9250,1014
3,3,422,0,65,70,3,1,35.0,1,0,53.1000,2
4,4,22,1,649,185,3,3,35.0,0,0,8.0500,1014


In [115]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df['Cabin_hash'] = df['Cabin'].map(lambda x: hash(x)%10)
df = df.drop('Cabin_count', axis=1)
df.head()

,index,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_hash
0,0,155,1,720,185,3,3,22.0,1,0,7.2500,5
1,1,286,0,816,106,0,1,38.0,1,0,71.2833,6
2,2,523,0,914,185,3,3,26.0,0,0,7.9250,5
3,3,422,0,65,70,3,1,35.0,1,0,53.1000,0
4,4,22,1,649,185,3,3,35.0,0,0,8.0500,5


In [116]:
train_X = df[:train_num]
print(f'Score is: {cross_val_score(estimator,train_X,train_Y,cv=5).mean()}')
train_X.head()

C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\log

Score is: 0.7699426942531474


,index,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_hash
0,0,155,1,720,185,3,3,22.0,1,0,7.2500,5
1,1,286,0,816,106,0,1,38.0,1,0,71.2833,6
2,2,523,0,914,185,3,3,26.0,0,0,7.9250,5
3,3,422,0,65,70,3,1,35.0,1,0,53.1000,0
4,4,22,1,649,185,3,3,35.0,0,0,8.0500,5


In [117]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df = pd.merge(df, cabin_count, on=['Cabin'], how='left')
train_X = df[:train_num]
print(f'Score is: {cross_val_score(estimator,train_X,train_Y,cv=5).mean()}')
train_X.head()

C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\cptien\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\log

Score is: 0.7700058195885008


,index,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_hash,Cabin_count
0,0,155,1,720,185,3,3,22.0,1,0,7.2500,5,1014
1,1,286,0,816,106,0,1,38.0,1,0,71.2833,6,2
2,2,523,0,914,185,3,3,26.0,0,0,7.9250,5,1014
3,3,422,0,65,70,3,1,35.0,1,0,53.1000,0,2
4,4,22,1,649,185,3,3,35.0,0,0,8.0500,5,1014
